In [1]:
from sqlalchemy import create_engine

engine = create_engine("sqlite+pysqlite:///:memory:", echo=True)

In [2]:
from sqlalchemy import text

with engine.begin() as conn:
    conn.execute(text("CREATE TABLE some_table (x int, y int)"))
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 1, "y": 1}, {"x": 2, "y": 4}]
    )


2023-12-24 13:50:11,550 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-24 13:50:11,550 INFO sqlalchemy.engine.Engine CREATE TABLE some_table (x int, y int)
2023-12-24 13:50:11,552 INFO sqlalchemy.engine.Engine [generated in 0.00089s] ()
2023-12-24 13:50:11,553 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2023-12-24 13:50:11,554 INFO sqlalchemy.engine.Engine [generated in 0.00191s] [(1, 1), (2, 4)]
2023-12-24 13:50:11,554 INFO sqlalchemy.engine.Engine COMMIT


In [4]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT x, y FROM some_table"))
    for x, y in result:
        print(x, y)

2023-12-24 13:52:05,216 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-24 13:52:05,217 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table
2023-12-24 13:52:05,217 INFO sqlalchemy.engine.Engine [cached since 47.11s ago] ()
1 1
2 4
2023-12-24 13:52:05,217 INFO sqlalchemy.engine.Engine ROLLBACK


In [5]:
from sqlalchemy.orm import Session

stmt = text("SELECT x, y FROM some_table")
with Session(engine) as session:
    result = session.execute(stmt)
    for row in result:
        print(row.x, row.y)

2023-12-24 13:57:04,020 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-24 13:57:04,028 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table
2023-12-24 13:57:04,028 INFO sqlalchemy.engine.Engine [cached since 345.9s ago] ()
1 1
2 4
2023-12-24 13:57:04,028 INFO sqlalchemy.engine.Engine ROLLBACK


In [6]:
from sqlalchemy import MetaData
metadata_obj = MetaData()

from sqlalchemy import Table, Column, Integer, String
user_table = Table(
    "user_account",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("name", String(30)),
    Column("fullname", String)
)

In [ ]:
from sqlalchemy import ForeignKey
address_table = Table(
    "address",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("user_id", ForeignKey("user_account.id"), nullable=False),
    Column("email_address", String, nullable=False)
)

metadata_obj.create_all(engine)

In [17]:
from sqlalchemy.orm import (DeclarativeBase, 
                            Mapped,
                            mapped_column, 
                            relationship)
from typing import Optional, List

class Base(DeclarativeBase):
    pass 

class User(Base):
    __tablename__ = "user_account"

    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[Optional[str]] 
    adresses: Mapped[List["Address"]] = relationship("Address", back_populates="user")

    def __repr__(self) -> str:
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"


class Address(Base):
    __tablename__ = "address"

    id: Mapped[int] = mapped_column(primary_key=True)
    email_address: Mapped[str]
    user_id: Mapped[int] = mapped_column(ForeignKey("user_account.id"), nullable=False)
    user: Mapped[User] = relationship("User", back_populates="addresses")

    def __repr__(self) -> str:
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"
    
metadata_obj.create_all(engine)

2023-12-24 15:04:07,223 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-24 15:04:07,223 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2023-12-24 15:04:07,223 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-12-24 15:04:07,226 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2023-12-24 15:04:07,226 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-12-24 15:04:07,227 INFO sqlalchemy.engine.Engine COMMIT


In [18]:
from sqlalchemy import insert

stmt = insert(user_table).values(name="spongebob", fullname="Spongebob Squarepants")
print(stmt)

INSERT INTO user_account (name, fullname) VALUES (:name, :fullname)
